##### Copyright 2019 The [TensorFlow](https://www.tensorflow.org/text/guide/subwords_tokenizer) Authors.


Пархоменко Александр

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Токенизаторы на уровне слов

В этом руководстве показано, как создать словарь из набора данных, используя [`text.BertTokenizer`](https://www.tensorflow.org/text/api_docs/python/text/BertTokenizer).

Основным преимуществом токенизатора уровня слов является то, что он выполняет интерполяцию между словарной и символьной разметкой. Обычные слова получают место в словарном запасе, но токенизатор может вернуться к частям слова и отдельным символам для неизвестных слов.

Цель: В конце этого урока вы построите полный уровня слова токенизатор от начала до конца и детокенизатор с нуля, и сохраните его как saved_model, вы сможете загружать и использовать его в руководстве по [трансформерам](https://tensorflow.org/text/tutorials/transformer).

\* токены = лексемы

## Обзор

Пакет `tensorflow_text` включает в себя реализацию TensorFlow многих распространенных токенизаторов. Сюда входят три токенизатора на уровне слов:

* [`text.BertTokenizer`](https://www.tensorflow.org/text/api_docs/python/text/BertTokenizer) - класс является высокоуровневым интерфейсом. Он включает в себя алгоритм расщепления на BERT-токены и `WordPieceTokenizer`. Он принимает **предложения** в качестве входных данных и возвращает **токен-идентификаторы**.
* `text.WordpieceTokenizer` - класс является низкоуровневым интерфейсом. Он только реализует [WordPiece алгоритм](#applying_wordpiece). Перед вызовом необходимо стандартизировать и разбить текст на слова. Он принимает **слова** в качестве входных данных и возвращает токен-идентификаторы.
* [`text.SentencepieceTokenizer`](https://www.tensorflow.org/text/api_docs/python/text/SentencepieceTokenizer) - требует более сложной настройки. Для его инициализатора требуется предварительно обученная уровня предложений модель. См [google/sentencepiece repository](https://github.com/google/sentencepiece#train-sentencepiece-model) для получения инструкций о том , как построить одну из этих моделей. Он может принимать **предложения** в качестве входных данных при токенизации.

В этом руководстве словарь Wordpiece создается сверху вниз, начиная с существующих слов. Этот процесс не работает для японского, китайского или корейского языков, поскольку в этих языках нет четких многосимвольных единиц. Чтобы разметить эти языки рассмотрите возможность использования `text.SentencepieceTokenizer`, `text.UnicodeCharTokenizer` или [этот подход](https://tfhub.dev/google/zh_segmentation/1). 

## Настройка

In [2]:
#!pip install -q -U "tensorflow-text==2.8.*"

In [3]:
#!pip install -q tensorflow_datasets

In [4]:
import collections
import os
import pathlib
import re
import string
import sys
import tempfile
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow as tf

In [5]:
tf.get_logger().setLevel('ERROR')
pwd = pathlib.Path.cwd()

## Загрузка набора данных

Скачиваем Русский/English датасет переводов из [tfds](https://tensorflow.org/datasets):

In [6]:
examples, metadata = tfds.load('ted_hrlr_translate/ru_to_en', with_info=True,
                               as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']  

2022-08-23 16:15:27.208900: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-23 16:15:27.209552: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Этот набор данных создает пары предложений на русском и английском языках:

In [7]:
for ru, en in train_examples.take(1):
  print("Russian: ", ru.numpy().decode('utf-8'))
  print("English:   ", en.numpy().decode('utf-8'))
# TODO: улучшить пример

Russian:  к : успех , перемены возможны только с оружием в руках .
English:    c : success , the change is only coming through the barrel of the gun .


2022-08-23 16:15:27.335031: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Обратите внимание на несколько моментов в приведенных выше примерах предложений:
* Они строчные.
* Вокруг знаков препинания есть пробелы.
* Неясно, используется ли нормализация Unicode и какая.

In [8]:
train_en = train_examples.map(lambda ru, en: en)
train_ru = train_examples.map(lambda ru, en: ru)

## Создание словаря

В этом разделе создается словарный запас из набора данных. Если у вас уже есть файл словаря и вы просто хотите увидеть, как построить `text.BertTokenizer` или `text.WordpieceTokenizer` с ним, то вы можете перейти вперед к разделу [Инициализация токенизатора](#build_the_tokenizer).

Примечание: Код генерации словаря, используемый в этом руководстве оптимизирован для **простоты**. Если вам нужно более масштабируемое решение рассмотрите вопрос об использовании реализации Apache Beam, доступной в [tools/wordpiece_vocab/generate_vocab.py](https://github.com/tensorflow/text/blob/master/tensorflow_text/tools/wordpiece_vocab/generate_vocab.py)

Код генерации словаря входит в `tensorflow_text` pip пакет. По умолчанию он не импортируется, его нужно импортировать вручную:

In [9]:
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

Функция `bert_vocab.bert_vocab_from_dataset` будет генерировать словарь. 

Вы можете задать множество аргументов, чтобы изменить её поведение. В этом руководстве вы в основном будете использовать значения по умолчанию. Если вы хотите узнать больше о возможностях, сначала прочтите раздел [алгоритм](#algorithm), а затем посмотреть на [код](https://github.com/tensorflow/text/blob/master/tensorflow_text/tools/wordpiece_vocab/bert_vocab_from_dataset.py).

Это займет около 2 минут.

In [10]:
bert_tokenizer_params=dict(lower_case=True)
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

bert_vocab_args = dict(
    # Размер целевого словаря
    vocab_size = 8000,
    # Зарезервированные токены, которые должны быть включены в словарь
    reserved_tokens=reserved_tokens,
    # Аргументы для `text.BertTokenizer`
    bert_tokenizer_params=bert_tokenizer_params,
    # Аргументы для `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
    learn_params={},
)

In [11]:
%%time
ru_vocab = bert_vocab.bert_vocab_from_dataset(
    train_ru.batch(1000).prefetch(2),
    **bert_vocab_args
)

CPU times: user 8min 39s, sys: 5.37 s, total: 8min 44s
Wall time: 8min 37s


Вот несколько кусочков получившейся лексики.

In [12]:
print(ru_vocab[:10])
print(ru_vocab[100:110])
print(ru_vocab[1000:1010])
print(ru_vocab[-10:])

['[PAD]', '[UNK]', '[START]', '[END]', '!', '#', '$', '%', '&', "'"]
['ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'і', '՛']
['трудно', 'хотела', 'далеко', 'качестве', 'мою', '##3', '##де', '##ила', 'планеты', 'большие']
['##’', '##“', '##”', '##„', '##•', '##′', '##⁄', '##∇', '##♪', '##♫']


Сохранение словарного файла:

In [13]:
def write_vocab_file(filepath, vocab):
  with open(filepath, 'w') as f:
    for token in vocab:
      print(token, file=f)

In [14]:
write_vocab_file('ru_vocab.txt', ru_vocab)

Используем эту функцию для создания словаря из английских данных:

In [15]:
%%time
en_vocab = bert_vocab.bert_vocab_from_dataset(
    train_en.batch(1000).prefetch(2),
    **bert_vocab_args
)


CPU times: user 1min 54s, sys: 1.36 s, total: 1min 55s
Wall time: 1min 53s


In [16]:
print(en_vocab[:10])
print(en_vocab[100:110])
print(en_vocab[1000:1010])
print(en_vocab[-10:])

['[PAD]', '[UNK]', '[START]', '[END]', '!', '#', '$', '%', '&', "'"]
['##s', 'have', 'but', 'what', 'on', 'do', 'with', 'can', 'there', 'about']
['revolution', '200', 'basic', 'potential', 'english', 'led', 'message', 'perfect', '##ce', 'nine']
['##–', '##—', '##‘', '##’', '##“', '##”', '##•', '##∇', '##♪', '##♫']


Вот два файла словаря:

In [17]:
write_vocab_file('en_vocab.txt', en_vocab)

In [18]:
!ls *_vocab.txt

en_vocab.txt  ru_vocab.txt


## Инициализация токенизатора
<a id="build_the_tokenizer"></a>

`text.BertTokenizer` можно инициализировать, передавая путь словарного файла в качестве первого аргумента (смотрите раздел [tf.lookup](#tf.lookup) для других вариантов):

In [19]:
ru_tokenizer = text.BertTokenizer('ru_vocab.txt', **bert_tokenizer_params)
en_tokenizer = text.BertTokenizer('en_vocab.txt', **bert_tokenizer_params)

Теперь вы можете использовать его для кодирования текста. Возьмите партию из 3 примеров из английских данных:

In [20]:
for ru_examples, en_examples in train_examples.batch(3).take(1):
  for ex in en_examples:
    print(ex.numpy())

b'c : success , the change is only coming through the barrel of the gun .'
b'the documentation and the hands-on teaching methodology is also open-source and released as the creative commons .'
b"( video ) didi pickles : it 's four o'clock in the morning ."


Запустите его через `BertTokenizer.tokenize` метод. Первоначально он возвращает `tf.RaggedTensor` с осями `(batch, word, word-piece)`:

In [21]:
# Tokenize the examples -> (batch, word, word-piece)
token_batch = en_tokenizer.tokenize(en_examples)
# Merge the word and word-piece axes -> (batch, tokens)
token_batch = token_batch.merge_dims(-2,-1)

for ex in token_batch.to_list():
  print(ex)

[41, 28, 1103, 14, 84, 243, 93, 200, 389, 218, 84, 6405, 87, 84, 2473, 16]
[84, 3914, 464, 85, 84, 702, 15, 104, 1495, 2346, 2024, 93, 187, 435, 15, 942, 85, 2533, 111, 84, 1068, 5725, 16]
[10, 400, 11, 168, 379, 1026, 1125, 28, 90, 9, 57, 316, 53, 9, 2501, 89, 84, 813, 16]


Если заменить токен-идентификаторы с текстовыми представлениями ( с помощью `tf.gather` ) вы можете увидеть, что в первом примере слова `"searchability"` и `"serendipity"` были разложенного на `"search ##ability"` и `"s ##ere ##nd ##ip ##ity"`:

In [22]:
# Lookup each token id in the vocabulary.
txt_tokens = tf.gather(en_vocab, token_batch)
# Join with spaces.
tf.strings.reduce_join(txt_tokens, separator=' ', axis=-1)

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'c : success , the change is only coming through the barrel of the gun .',
       b'the document ##ation and the hands - on teaching method ##ology is also open - source and released as the creative commons .',
       b"( video ) did ##i pick ##les : it ' s four o ' clock in the morning ."],
      dtype=object)>

Для того, чтобы повторно собрать слова из извлеченных токенов, используйте метод `BertTokenizer.detokenize`:

In [23]:
words = en_tokenizer.detokenize(token_batch)
tf.strings.reduce_join(words, separator=' ', axis=-1)

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'c : success , the change is only coming through the barrel of the gun .',
       b'the documentation and the hands - on teaching methodology is also open - source and released as the creative commons .',
       b"( video ) didi pickles : it ' s four o ' clock in the morning ."],
      dtype=object)>

> Примечание: `BertTokenizer.tokenize`/`BertTokenizer.detokenize` не совершает преобразование туда и обратно без потерь. Результат `detokenize`, как правило, не будет иметь того же содержимого или смещений, что и ввод для `tokenize`. Это происходит потому , что на стадии «базовой токенизации», которая разделяет строки на слова перед применением `WordpieceTokenizer`, включает в себя необратимые шаги, как нижний регистр и расщепление на знаки препинания. `WordpieceTokenizer` с другой стороны, **является** обратимым.

## Настройка и экспорт

Это руководство строит текстовый токенайзер и детокенайзер используемый в руководстве [Transformer](https://tensorflow.org/text/tutorials/transformer). Этот раздел добавляет методы и этапы обработки, чтобы упростить это руководство, а также экспорт токенайзера с помощью `tf.saved_model` таким образом они могут быть импортированы в других руководствах.

### Настройка токенизатора

Ниже в руководстве ожидаются в токенизированном тексте включение двух `[START]` и `[END]` токенов.

`reserved_tokens` резервируют место в начале словаря, так `[START]` и `[END]` имеют одинаковые индексы для обоих языков:

In [24]:
START = tf.argmax(tf.constant(reserved_tokens) == "[START]")
END = tf.argmax(tf.constant(reserved_tokens) == "[END]")

def add_start_end(ragged):
  count = ragged.bounding_shape()[0]
  starts = tf.fill([count,1], START)
  ends = tf.fill([count,1], END)
  return tf.concat([starts, ragged, ends], axis=1)

In [25]:
words = en_tokenizer.detokenize(add_start_end(token_batch))
tf.strings.reduce_join(words, separator=' ', axis=-1)

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'[START] c : success , the change is only coming through the barrel of the gun . [END]',
       b'[START] the documentation and the hands - on teaching methodology is also open - source and released as the creative commons . [END]',
       b"[START] ( video ) didi pickles : it ' s four o ' clock in the morning . [END]"],
      dtype=object)>

### Настройка детокенизатора

Перед экспортом токенизаторов есть несколько вещей, которые вы можете прояснить для последующих руководств:

1. Мы хотим генерировать чистый вывод текста, так что отбрасываем зарезервированные лексемы, как `[START]`, `[END]` и `[PAD]`.
2. Мы заинтересованы в полных строках, поэтому применяем строчное присоединение по `words` оси результата.  

In [26]:
def cleanup_text(reserved_tokens, token_txt):
  # Отбрасываем зарезервированные токены, кроме "[UNK]".
  bad_tokens = [re.escape(tok) for tok in reserved_tokens if tok != "[UNK]"]
  bad_token_re = "|".join(bad_tokens)
    
  bad_cells = tf.strings.regex_full_match(token_txt, bad_token_re)
  result = tf.ragged.boolean_mask(token_txt, ~bad_cells)

  # Соединяем их в строки.
  result = tf.strings.reduce_join(result, separator=' ', axis=-1)

  return result

In [27]:
en_examples.numpy()

array([b'c : success , the change is only coming through the barrel of the gun .',
       b'the documentation and the hands-on teaching methodology is also open-source and released as the creative commons .',
       b"( video ) didi pickles : it 's four o'clock in the morning ."],
      dtype=object)

In [28]:
token_batch = en_tokenizer.tokenize(en_examples).merge_dims(-2,-1)
words = en_tokenizer.detokenize(token_batch)
words

<tf.RaggedTensor [[b'c', b':', b'success', b',', b'the', b'change', b'is', b'only',
  b'coming', b'through', b'the', b'barrel', b'of', b'the', b'gun', b'.'],
 [b'the', b'documentation', b'and', b'the', b'hands', b'-', b'on',
  b'teaching', b'methodology', b'is', b'also', b'open', b'-', b'source',
  b'and', b'released', b'as', b'the', b'creative', b'commons', b'.']    ,
 [b'(', b'video', b')', b'didi', b'pickles', b':', b'it', b"'", b's',
  b'four', b'o', b"'", b'clock', b'in', b'the', b'morning', b'.']    ]>

In [29]:
cleanup_text(reserved_tokens, words).numpy()

array([b'c : success , the change is only coming through the barrel of the gun .',
       b'the documentation and the hands - on teaching methodology is also open - source and released as the creative commons .',
       b"( video ) didi pickles : it ' s four o ' clock in the morning ."],
      dtype=object)

### Экспорт

Следующий блок кода создает `CustomTokenizer` класс, чтобы содержать `text.BertTokenizer` экземпляры, пользовательскую логику, и `@tf.function` обертки, необходимых для экспорта. 

In [30]:
class CustomTokenizer(tf.Module):
  def __init__(self, reserved_tokens, vocab_path):
    self.tokenizer = text.BertTokenizer(vocab_path, lower_case=True)
    self._reserved_tokens = reserved_tokens
    self._vocab_path = tf.saved_model.Asset(vocab_path)

    vocab = pathlib.Path(vocab_path).read_text().splitlines()
    self.vocab = tf.Variable(vocab)

    ## Create the signatures for export:   

    # Include a tokenize signature for a batch of strings. 
    self.tokenize.get_concrete_function(
        tf.TensorSpec(shape=[None], dtype=tf.string))
    
    # Include `detokenize` and `lookup` signatures for:
    #   * `Tensors` with shapes [tokens] and [batch, tokens]
    #   * `RaggedTensors` with shape [batch, tokens]
    self.detokenize.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64))
    self.detokenize.get_concrete_function(
          tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

    self.lookup.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64))
    self.lookup.get_concrete_function(
          tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

    # These `get_*` methods take no arguments
    self.get_vocab_size.get_concrete_function()
    self.get_vocab_path.get_concrete_function()
    self.get_reserved_tokens.get_concrete_function()
    
  @tf.function
  def tokenize(self, strings):
    enc = self.tokenizer.tokenize(strings)
    # Merge the `word` and `word-piece` axes.
    enc = enc.merge_dims(-2,-1)
    enc = add_start_end(enc)
    return enc

  @tf.function
  def detokenize(self, tokenized):
    words = self.tokenizer.detokenize(tokenized)
    return cleanup_text(self._reserved_tokens, words)

  @tf.function
  def lookup(self, token_ids):
    return tf.gather(self.vocab, token_ids)

  @tf.function
  def get_vocab_size(self):
    return tf.shape(self.vocab)[0]

  @tf.function
  def get_vocab_path(self):
    return self._vocab_path

  @tf.function
  def get_reserved_tokens(self):
    return tf.constant(self._reserved_tokens)

Построение `CustomTokenizer` для каждого языка:

In [31]:
tokenizers = tf.Module()
tokenizers.ru = CustomTokenizer(reserved_tokens, 'ru_vocab.txt')
tokenizers.en = CustomTokenizer(reserved_tokens, 'en_vocab.txt')

Экспорт токенайзеров как `saved_model`:

In [32]:
model_name = 'ted_hrlr_translate_ru_en_converter'
tf.saved_model.save(tokenizers, model_name)

Обновление `saved_model` и проверка методов:

In [33]:
reloaded_tokenizers = tf.saved_model.load(model_name)
reloaded_tokenizers.en.get_vocab_size().numpy()

7796

In [34]:
tokens = reloaded_tokenizers.en.tokenize(['Hello TensorFlow!'])
tokens.numpy()

array([[   2, 3372, 2214,  691,  952, 2669,    4,    3]])

In [35]:
text_tokens = reloaded_tokenizers.en.lookup(tokens)
text_tokens

<tf.RaggedTensor [[b'[START]', b'hello', b'tens', b'##or', b'##f', b'##low', b'!',
  b'[END]']]>

In [36]:
round_trip = reloaded_tokenizers.en.detokenize(tokens)

print(round_trip.numpy()[0].decode('utf-8'))

hello tensorflow !


Архивирование для [руководств по переводу](https://tensorflow.org/text/tutorials/transformer):

In [37]:
print(model_name) # ted_hrlr_translate_ru_en_converter

ted_hrlr_translate_ru_en_converter


In [38]:
!zip -r {model_name}.zip {model_name}

updating: ted_hrlr_translate_ru_en_converter/ (stored 0%)
updating: ted_hrlr_translate_ru_en_converter/saved_model.pb (deflated 91%)
updating: ted_hrlr_translate_ru_en_converter/variables/ (stored 0%)
updating: ted_hrlr_translate_ru_en_converter/variables/variables.index (deflated 33%)
updating: ted_hrlr_translate_ru_en_converter/variables/variables.data-00000-of-00001 (deflated 59%)
updating: ted_hrlr_translate_ru_en_converter/assets/ (stored 0%)
updating: ted_hrlr_translate_ru_en_converter/assets/ru_vocab.txt (deflated 70%)
updating: ted_hrlr_translate_ru_en_converter/assets/en_vocab.txt (deflated 54%)


In [39]:
!du -h *.zip

184K	ted_hrlr_translate_ru_en_converter.zip


<a id="algorithm"></a>

## Дополнительно: алгоритм


Здесь стоит отметить, что существует две версии алгоритма WordPiece: снизу вверх и сверху вниз. В обоих случаях цель одна и та же: "Учитывая обучающий корпус и количество желаемых токенов D, задача оптимизации состоит в том, чтобы выбрать D словесных частей таким образом, чтобы результирующий корпус был минимальным по количеству частей слова при сегментировании в соответствии с выбранной моделью словаря."

Оригинальный алгоритм [WordPiece снизу вверх](https://static.googleusercontent.com/media/research.google.com/ja//pubs/archive/37842.pdf), основан на [кодировании байт-пар](https://towardsdatascience.com/byte-pair-encoding-the-dark-horse-of-modern-nlp-eb36c7df4f10). Как и BPE, он начинается с алфавита и итеративно комбинирует общие биграммы для образования частей и слов.

Генератор словаря TensorFlow Text следует нисходящеё реализации из [BERT](https://arxiv.org/pdf/1810.04805.pdf). Начинает со слов и разбивает их на более мелкие компоненты, пока они не достигнут порога частоты или не могут быть разбиты дальше. В следующем разделе это подробно описано. Для японского, китайского и корейского языков этот подход сверху вниз не работает, поскольку нет явных единиц слова, с которых можно было бы начать. Для этого вам нужен [другой подход](https://tfhub.dev/google/zh_segmentation/1).

### Выбор словаря

WordPiece алгоритм генерации cверху вниз берет в наборе (слово, количество) пары и порог `T` и возвращает словарь `V`.

Алгоритм итерационный. Он выполняет `k` итераций, где обычно`k = 4`, но только первые два действительно важны. Третий и четвертый (и последующие) просто идентичны второму. Обратите внимание, что каждый шаг алгоритма двоичного поиска работает с нуля для `k` итераций.

Итерации описанны ниже:


#### Первая итерация

1.  Итерация по каждой (слово и счетчик) паре на входе, обозначенной как `(w, c)`.
2.  Для каждого слова `w`, генерируется подстрока, обозначаемая `s`. Например, для слова `human`, мы генерируем 
    `{h, hu, hum, huma, human, ##u, ##um, ##uma, ##uman, ##m, ##ma, ##man, #a, ##an, ##n}`.
3.  Ведем хеш-карту подстрока-в-счетчик, и увеличиваем счетчик `c` для каждой `s`. 
    Например, если мы имеем `(human, 113)` и `(humas, 3)` на нашем входе, счетчик 
    `s = huma` будет `113+3=116`.
4.  После того, как мы собрали значения счетчика для каждой подстроки, перебираем 
    `(s, c)` пары *начиная с самой длинной `s`*.
5.  Храним любую `s` что имеет `c > T`. Например: если `T = 100` и мы имеем 
    `(pers, 231); (dogs, 259); (##rint; 76)`, тогда мы будем хранили `pers` и `dogs`.
6.  Когда `s` сохранена, вычитаем её счетчик из всех её префиксов. 
    Это является причиной для сортировки всех `s` по длине в шаге 4. Это критическая часть алгоритма, 
    поскольку в противном случае слова будут подсчитаны дважды.
     Например, допустим , что мы сохранили `human` и мы получаем
    `(huma, 116)`. Мы знаем, что `113` из этих `116` пришли от `human`, и `3`
    пришли от `humas`. Однако теперь, когда `human` находится в нашем словаре, мы знаем
    что мы никогда не сегментируем `human` в `huma ##n`. Так как `human` был сохранен,
    то `huma` только имеет *эффективный* счетчик `3`.

Этот алгоритм будет генерировать набор кусочков слов `s` (многие из которых будет целыми словами `w`), которые мы *могли* бы использовать в качестве нашего WordPiece словаря.


Therefore, if we keep the word `human`, we will subtract off the count for `h,
hu, hu, huma`, but not for `##u, ##um, ##uma, ##uman` and so on. So we might
generate both `human` and `##uman` as word pieces, even though `##uman` will
never be applied.

Однако есть проблема: этот алгоритм будет сильно генерировать фрагменты слов. Причина в том, что мы вычитаем только счетчики токенов префикса. Поэтому, если мы держим слово `human`, мы вычитаем из счетчика для `h, hu, hu, huma` , но не для `##u, ##um, ##uma, ##uman` и так далее. Таким образом, мы могли бы генерировать как `human` и `##uman` как куски слов, даже если `##uman` никогда не будет применяться.

Так почему бы не вычитать из счетчиков для каждой *подстроки*, а не только каждый *префикс*? Потому что тогда мы могли бы вычесть счетчик несколько раз. Допустим, что мы обрабатываем `s` длины 5, и мы продолжаем как `(##denia, 129)` и `(##eniab, 137)`, где `65` из этих подсчетов произошло от слова `undeniable`. Если вычесть из *каждого* подстроки, мы вычитаем `65` из подстроки `##enia` дважды, несмотря на то, что мы должны только вычесть один раз. Однако, если мы будем вычитать только из префиксов, оно будет правильно вычтено только один раз.

#### Вторая (и третья ...) итерация

Чтобы решить проблему избыточной генерации, упомянутую выше, мы выполняем несколько итераций алгоритма.

Последующие итерации идентичны первому, но с одним важным отличием: в шаге 2, вместо того, чтобы рассматривать *каждую* подстроку, мы применяем алгоритм WordPiece лексемизации с использованием словаря из предыдущей итерации, и рассматривать только подстроки, которые *начинаются* на точках разделения.

Например, допустим , что мы выполняем шаг 2 алгоритма и сталкиваются слово `undeniable`. 
В первой итерации, мы будем рассматривать каждую подстроку, например,
`{u, un, und, ..., undeniable, ##n, ##nd, ..., ##ndeniable, ...}`.

Теперь, для второй итерации, мы рассмотрим только их подмножество. Предположим, что после первой итерации соответствующие части слова:

`un, ##deni, ##able, ##ndeni, ##iable`

Алгоритм WordPiece сегментирует это в `un ##deni ##able` (смотрите раздел [Применение WordPiece](#applying-wordpiece) для получения дополнительной информации). В этом случае мы будем рассматривать только подстроки, которые *начинаются* в точке сегментации. 
Мы еще рассмотрим каждое возможное *конечное* положение. Таким образом, во второй итерации, множество `s` для `undeniable` является:

`{u, un, und, unden, undeni, undenia, undeniab, undeniabl,
undeniable, ##d, ##de, ##den, ##deni, ##denia, ##deniab, ##deniabl
, ##deniable, ##a, ##ab, ##abl, ##able}`

В остальном алгоритм идентичен. В этом примере, в первой итерации, алгоритм производит ложные токены `##ndeni` и `##iable`. Теперь эти токены никогда не учитываются, поэтому они не будут сгенерированы на второй итерации. Мы выполняем несколько итераций, чтобы убедиться, что результаты сходятся (хотя буквальной гарантии сходимости нет).

### Применение WordPiece

<a id="applying_wordpiece"></a>

После создания словаря WordPiece нам нужно иметь возможность применять его к новым данным. Алгоритм представляет собой простое «жадное» приложение с поиском наиболее длинного совпадения.

Например, рассмотрим сегментирование слово `undeniable`.

We first lookup `undeniable` in our WordPiece dictionary, and if it's present,
we're done. If not, we decrement the end point by one character, and repeat,
e.g., `undeniabl`.

Сначала ищем `undeniable` в нашем WordPiece словаре, и если оно присутствует, готово. Если нет, то мы уменьшаем конечную точку на один символ, и повторяем, например, `undeniabl` .

В конце концов, мы либо найдем подтокен в нашем словаре, либо перейдем к подтокену, состоящему из одного символа. (В целом, мы считаем, что каждый символ в нашем словаре, хотя это может быть не так для редких символов Unicode. Если мы встречаем редкий символ Unicode, что не в словаре, мы просто отображаем всё слово `<unk>`).

В этом случае, мы находим `un` в нашем словаре. Итак, это наш первый отрывок из слов. Затем мы переходим к концу `un` и повторяем обработку, например, попытаемся найти `##deniable`, затем `##deniabl` и т.д. Это повторяется до тех пор, пока не сегментируем слово целиком.

### Объяснение

Интуитивно понятно, что токенизация уровня сслова пытается решить две разные задачи:

1.  Токенизировать данные в наименьшее число частей, как это возможно. Важно помнить, что алгоритм WordPiece не «хочет» разбивать слова. Иначе, он бы просто разделил каждое слово на символы, например,  `люди -> {л, ##ю, ##д, #и}`. Это критическая вещь, которая делает WordPiece отличным от морфологических разветвителей, которые разделят лингвистические морфемы даже для общих слов (например: `любимые -> {лю, бим, ые}`).

2.  Когда слово действительно нужно разбить на части, разбейте его на части, которые
    имеют максимальный счетчик в обучающих данных. Например, причина , почему слово
    `undeniable` будет разбито на `{un, ##deni, ##able}` в отличии от альтернативы `{unde, ##niab, ##le}` является то, что счетчики для `un` и `##able` в частности, будет очень высоким, так как это общие префиксы и суффиксы. Даже несмотря на то, счетчик для `##le` должен быть выше , чем `##able` , низкие счетчики `unde` и `##niab` сделает это менее «желательной» токенизацией для алгоритма.

## Дополнительно: tf.lookup

<a id="tf.lookup"></a>

Если вам нужен доступ или больше контроля над словарем стоит отметить, что вы можете создать таблицу поиска сами и передать в `BertTokenizer`.

Когда вы передаете строку, `BertTokenizer` выполняет следующие действия:

In [40]:
ru_lookup = tf.lookup.StaticVocabularyTable(
    num_oov_buckets=1,
    initializer=tf.lookup.TextFileInitializer(
        filename='ru_vocab.txt',
        key_dtype=tf.string,
        key_index = tf.lookup.TextFileIndex.WHOLE_LINE,
        value_dtype = tf.int64,
        value_index=tf.lookup.TextFileIndex.LINE_NUMBER)) 
ru_tokenizer = text.BertTokenizer(ru_lookup)

Теперь у вас есть прямой доступ к таблице поиска, используемой в токенизаторе.

In [41]:
ru_lookup.lookup(tf.constant(['é', 'um', 'uma', 'para', 'não']))

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([7832, 7832, 7832, 7832, 7832])>

Вам не нужно использовать словарный файл, `tf.lookup` имеет другие параметры инициализатора. Если у вас есть словарный запас в памяти можно использовать `lookup.KeyValueTensorInitializer`:

In [42]:
ru_lookup = tf.lookup.StaticVocabularyTable(
    num_oov_buckets=1,
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=ru_vocab,
        values=tf.range(len(ru_vocab), dtype=tf.int64))) 
ru_tokenizer = text.BertTokenizer(ru_lookup)

Конец